## 1 The cliff problem (3 pts.)

Consider the following problem. An agent must navigate the grid world represented in Fig. 1, where the
grey area corresponds to a cliff that the agent must avoid. The goal state corresponds to the cell marked
with a G, while the cell marked with an S corresponds to a starting state. At every step, the agent receives
a reward of −1 except in the cliff region, where the reward is −100. Whenever the agent steps into a cliff
state, its position is reset back to the start state. When the agent steps into the goal state, the episode ends.
The agent has available four actions: up (U), down (D), left (L) and right (R), all of which move the agent
deterministically in the corresponding direction.

![image.info](./pictures/grid-world.png)

In this question, you will compare the performance of SARSA and Q-learning on the cliff task. To do
so, assume that the agent follows an ε-greedy policy, with ε = 0.15. Run both algorithms for 500 episodes, making sure that the Q-values for both methods are initialized to 0. Consider throughout that γ = 1 and use a step-size of α = 0.5.


### Question 1. Compare:
• The total reward in each episode for Q-learning and SARSA, plotting the two in a single plot.<br>
• The resulting policy after the 500 episodes.<br><br>
Comment any differences observed.<br><br>
<b>Note<b>: To mitigate the effect of noise on the plot, perform multiple runs and average the result across
runs.

## Sarsa pseudo code

![image.info](./pictures/sarsa.png)

In [22]:
# Sarsa Implementation

class SARSACliff(object):
    
    def __init__(self, gamma=1):
        
        # Action space: A = {up, down, left, right}
        self.actions = [0, 1, 2, 3]

        # Grid space - available states: S = {4*11 matrix}
        self.states = np.zeros((4, 11))
        
        # Q-values: arbitrarily set to 1, except for terminal state, where it will be zero
        self.q_up = np.zeros((4, 11)) + 1
        self.q_up[3, 10] = 0
        
        self.q_down = np.zeros((4, 11)) + 1
        self.q_down[3, 10] = 0
        
        self.q_left = np.zeros((4, 11)) + 1
        self.q_left[3, 10] = 0
        
        self.q_right = np.zeros((4, 11)) + 1
        self.q_right[3, 10] = 0
        
        # Rewards: -1 for every action, except for cliff, where it is -100
        self.rewards = np.zeros((4, 11)) - 1
        self.rewards[3, 1:10] = -100 

        # Discount factor gamma
        self.gamma = gamma
        
        print(self.states)
        print(self.rewards)
        print(self.q_up)
        
        return
    
SARSACliff()

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[  -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.]
 [  -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.]
 [  -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.]
 [  -1. -100. -100. -100. -100. -100. -100. -100. -100. -100.   -1.]]
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.]]
